In [1]:
import time
start = time.time()

In [2]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-0e9503d3-ba14-6d92-2c7e-967f62035a22)


In [3]:
%%time

! pip install -qq -U langchain tiktoken pypdf chromadb faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers
! pip install -qq -U bitsandbytes bitsandbytes-cuda117

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cmudict 1.0.13 requires importlib-metadata<6.0.0,>=5.1.0, but you have importlib-metadata 6.0.1 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
dask-cudf 23.8.0 requires dask==2023.7.1, but you have dask 2023.9.0 which is incompatible.
dask-cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.2 which is incompatible.
distributed 2023.7.1 requires dask==2023.7.1, but yo

In [4]:
import warnings
warnings.filterwarnings("ignore")

import os
import textwrap
import shutil
import pandas as pd
import glob

import langchain
from langchain.llms import HuggingFacePipeline

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

#quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

#os.environ['TRANSFORMERS_CACHE'] = 'cache/'

print(langchain.__version__)

0.0.331


In [5]:
### Multi-document retriever
from langchain.vectorstores import Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA, VectorDBQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

In [6]:
%%time
print('\nDownloading model: ', '\n\n')
tokenizer = AutoTokenizer.from_pretrained("Photolens/llama-2-7b-langchain-chat", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("Photolens/llama-2-7b-langchain-chat",
                                             load_in_8bit=True,
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             low_cpu_mem_usage=True,
                                             #cache_dir="cache/",
                                             #offload_folder="save_folder"
#                                            rope_scaling = {'type': 'dynamic', 'factor': 2.0}
                                            )
max_len = 1024 # 512
task = "text-generation"
T = 0.1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 20.9 s, sys: 29.9 s, total: 50.8 s
Wall time: 6min 47s


In [7]:
pipe = pipeline(
    task=task,
    model=model, 
    tokenizer=tokenizer, 
    max_length=max_len,
    temperature=T,
    top_p=0.95,
    repetition_penalty=1.15,
    max_new_tokens=1024
)

llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7f0430ea6590>)

In [9]:
shutil.copytree('/kaggle/input/mkvc-finalmix/kaggle/working/mk-vectordb-chroma', '/kaggle/working/mk-vectordb-chroma')

'/kaggle/working/mk-vectordb-chroma'

In [10]:
%%time

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="intfloat/multilingual-e5-large", model_kwargs={"device": "cuda"})

vectordb = Chroma(persist_directory='mk-vectordb-chroma',embedding_function=instructor_embeddings,)
#vectordb.persist()

load INSTRUCTOR_Transformer
max_seq_length  512


ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/migration#migration-to-0416---november-7-2023 


In [11]:
%%time
retriever = vectordb.as_retriever(search_kwargs={"k": 3, "search_type" : "similarity"})
#retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 6, "lambda_mult": 0.25})

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                       chain_type="stuff", 
                                       retriever=retriever, 
                                       return_source_documents=True,
                                       verbose=False)

NameError: name 'vectordb' is not defined

In [12]:
qa_chain

NameError: name 'qa_chain' is not defined

In [ ]:
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
def llm_ans(query):
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    return ans

In [ ]:
model

In [ ]:
dataset = pd.read_csv('/kaggle/input/mkri-qa-3/dataset.csv',sep=';')
dataset

In [ ]:
%%time
for x,y in zip(dataset.no,dataset.qa_comb):
    llm_time_start = time.time()
    print("Nomor: ",x)
    print(y)
    llm_ans(y)
    llm_time_end = time.time()
    print("Time: ",llm_time_end - llm_time_start)
    print("==============================================")

In [ ]:
end = time.time()
print(end - start)